##### Module 4: Concurrency
Course: Advanced Programming for CSAI (Spring 2025)

Topics covered in this module:  
- Concurrent Programming.
- Threading and Multiprocessing.
- Communication between threads and processes.

**Warning**: some students found problems when running the multiprocessing library from within Jupyter Notebooks on Windows. The problems were solved when running the notebooks from JupyterLab: https://jupyterlab.uvt.nl. 

You are advised to work on this notebook after, or in parallel to, consulting other materials of the module, such as the slide deck and book chapters. The notebook contains examples and exercises that should help you understand and apply the concepts introduced in the rest of materials. You may also use the official Python docs: https://docs.python.org/3/.

Do not hesitate to be creative when trying out the examples: you can play with the code. You can try variants of the examples and exercises, print values of the variables to understand what is going on at every step, and come up with different solutions to the same exercise and think about relative advantages of each one.

The notebook also contains formative assignments. These are indicated as FA-n, where n is a number id. As explained on the course guide, you have to submit these. Please submit your best effort (*i.e.*, FA-n questions with no answer will be considered incomplete), and **if your solution does not work or you think it is inadequate, add a comment explaining why you could not proceed further**.

To submit the formative assignments, we ask you to upload the filled-in notebook. The notebook you upload should contain *at least* the formative assigments. It's not a problem if you upload the notebook with additional code, like the variants and tests mentioned above. However, to grade your assignments, we will only look at the answers to the requested exercises (those indicated with FA-n), so **make sure you store your answers in the corresponding variables and/or to name your functions as indicated**.

Optional exercises are, as the name indicates, not mandatory for the formative assignments. These are exercises that suggest you to create an alternative approach, or which propose a longer problem that allows for the integration of earlier concepts in one solution; in general, they present scenarios where you can be more creative. To make the most of the course, it is best to try them out and share your solutions on the Discussion Board, so that your peers can comment on them. You are also encouraged to comment on the exercises of your fellow students. This will help you sharpen your evaluation skills, which is a great asset in programming, as in turn this will help you devise more robust, efficient and maintainable solutions. 


### /!\ Before submitting your notebook

Please check it can be ran without errors! You can check this by pressing kernel --> restart and run all before submitting. If it does not run without errors, it is your **responsibility** to fix the problem either by resolving the bug in your code or by commenting it out along with a comment.


---

## Threads and Processes
### Example: Pot of Sweets

Let's start with an example. Imagine you want to have a program that adds one sweet to a pot of sweets every second. The program could be as follows:

In [1]:
import time

def addSweet(potOfSweets): 
    potOfSweets += 1
    return potOfSweets


potOfSweets= 0
runtime = 20

#returns the current time, so time at the start of the program
startTime = time.time()

#this loop runs for twenty seconds after the starting time
while(time.time()<startTime+runtime): 
    #and adds one sweet...
    potOfSweets = addSweet(potOfSweets)
    #then waits one second
    time.sleep(1)
    print("added a sweet")
    
    

added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet
added a sweet


Suppose you wanted to add a switch to this program so that you can control when sweets get added. If the switch is on, the sweets get added every second, but if it is off, no sweets get added. You want the user to control the switch by inserting "on" or "off" in the console. How would you implement this in the program above? 

You could try something like the following:

In [2]:
import time
from pcinput import getString #This file has been provided for you; please download it from Canvas

def addSweet(potOfSweets): 
    potOfSweets += 1
    return potOfSweets

potOfSweets= 0
runtime = 20

#returns the current time, so time at the start of the program
startTime = time.time()

switch = True


#this loop runs for 20 seconds after the starting time
while(time.time()<startTime+runtime): 
    switchPosition = getString("Insert switch position:") #asks the user what position the switch should have
    if switchPosition == "on": #so when this is complete and =True then it goes straight to the next if 
        switch = True
    elif switchPosition == "off":
        switch = False
    else:
        print("Unknown value {}. Insert values \"on\" or \"off\"".format(switchPosition))
        switch = False        
    if switch:
        potOfSweets = addSweet(potOfSweets)
        print("added a sweet")
    #then waits one second
    time.sleep(1)

print("Total amount of sweets: ", potOfSweets) #print the amount of sweets

Insert switch position: on


added a sweet


Insert switch position: off
Insert switch position: off
Insert switch position: stop


Unknown value stop. Insert values "on" or "off"


Insert switch position: end


Unknown value end. Insert values "on" or "off"


Insert switch position: hi


Unknown value hi. Insert values "on" or "off"
Total amount of sweets:  1


This program uses a polling approach: it actively and constantly queries for the status of the switch. It succeeds in its mission, but the fact that the program needs to wait for the user input at every time-step is not very efficient, specially when the user just wants to keep the switch on.

Ideally, we would prefer that the program keeps adding sweets while the switch is on, without having to wait for the user's input.  In other words, we would like that adding sweets is done *concurrently* to checking the switch.

## Solutions to concurrency

When you want different parts of your program running concurrently, you can choose between two ways of implementing this behavior: **threading** and **multiprocessing**. 

First, let's note that the potOfSweets has two main functionalities: adding candy to the pot (let's call it A), and keeping track of the switch (let's call it B). In the implementation above, they are executed in alternation, as ABABABAB...However, B cannot start until A has finished, and vice versa. 

With concurrent programming, we can do one of two things: either allow for A and B to alternate without the need that they wait for each other, or allow A and B to run fully in parallel. For the latter to happen, however, you need to have multiple cores. 

The first type of solution can be implemented with threads. In this case, you would define a thread for A and a thread for B. The threads would then run with interleaved computation: they take turns in using the CPU, but they do not have to wait for each other to finish (*i.e.*, we do not have to wait for the user to introduce information). As a programmer, you do not control which thread is being executed: this happens behind the curtains.

With a multiprocessing solution, we would define process A and process B. If there is a single CPU core, the processes will also take turns, without having to wait for each other to finish. However, if you have multiple cores, the processes could even run in parallel (*i.e.*, at the same time).

With threads, the use of multiple CPU cores is limited: the Global Interpreter Lock (GIL) in the Python interpreter makes sure **only one thread** can be executed at a time. Concurrency is therefore simulated by switching rapidly between threads. Thus, threading puts a ceiling on performance\*.  

Multiprocessing goes around the GIL by spawning an entirely different process. Effectively, your operating system sees each process as a different program, and schedules the CPU time as it normally does for any open program, taking advantage of multiple processors if available. However, one of the disadvantages of this approach is that there is no shared memory between processes, so exchanging data between processes is less efficient than between threads. Furthermore, the creation and destruction of processes has some overhead.

\* There are some exceptions, like when using Python libraries that are written in C (like TensorFlow, for example).

---

## Threading

### Simple implementation

Python provides the <code>threading</code> library to implement threading: https://docs.python.org/3/library/threading.html

An implementation of a thread can be done by calling the object constructor of a thread with a function as a keyword argument. 

Run this code and note the interleaved computation. You may see a different output when you run it again!

In [3]:
import threading
import time

#This is a simple function
def sayMessage(message, reps): 
    for _ in range(reps):      # `_` is commonly used as a throwaway variable, a placeholder in loops
        print(message)
        time.sleep(1)

    
#We create a thread to run our function
t1 = threading.Thread(target = sayMessage, #the target is the function that the thread will run
                      args = ("Hello, world!", 10)) #these are the arguments to such function

#start the thread
t1.start() 

print('We can run other instructions while the thread is alive.') #this is the 1st line of the Main thread


time.sleep(3) #pause the main program ; #while waiting the program executes the (side) threads 'hello world'


print('The thread keeps running while the main program sleeps.') #this is the 2nd line of the Main thread

#wait until the thread ends: https://docs.python.org/3/library/threading.html#threading.Thread.join
t1.join() #after join will never be printed until the whole process is done

#the following instruction is only printed after t1 has finished
print('end')


Hello, world!We can run other instructions while the thread is alive.

Hello, world!
Hello, world!
The thread keeps running while the main program sleeps.
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
end


We can create additional threads of the same function. Note the interleaved computation:

In [4]:
t1 = threading.Thread(target = sayMessage, args = ("Hello, world!", 10)) 
t2 = threading.Thread(target = sayMessage, args = ("Hello, all!", 4)) 

#here we dont have Main threads
t1.start() #start the thread
t2.start() #start a second thread
t1.join() #waits until thread ends
t2.join() #waits until thread ends

Hello, world!
Hello, all!
Hello, world!Hello, all!

Hello, all!
Hello, world!
Hello, world!Hello, all!

Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!


### FA-1:

Let's say we have a list of messages of unknown length, and we want to use the function sayMessage defined above for each one. We want the number of repetitions to be random, between 1 and 10. We want to create a thread for each call to the function.

Below there is a list of dry fruits, which we will use as an example of a message list. 

Write the missing code, such that:

1 - you create as many threads as messages in a list (in this case, **dry_fruits**) and save the threads in the **threads** list.

2 - after the threads have been created, start each one of them

3 - before the program finishes, you make sure all the threads have finished

**Hint**: each of these steps can be implemented in a very short **for** loop (although you are welcome to write your code with a different approach)

In [5]:
dry_fruits=['achene',
    'capsule',
    'caryopsis',
    'cypsela',
    'fibrous drupe',
    'follicle',
    'legume',
    'loment',
    'nut',
    'samara',
    'schizocarp',
    'silique',
    'silicle', 
    'utricle' ]

threads = []

#### FA-1 ####################################
import random
def sayMessage(message, reps): 
    for _ in range(reps): 
        print(message)
        time.sleep(1)

for word in dry_fruits:
    t = threading.Thread(target = sayMessage, args = (word, random.randint(1,10)) ) #each thread runs random number of times parallel to the other threads also with random number of times
    threads.append(t)

for thread in threads: #it starts all the 10 threads together and when they are done then it goes to join
    thread.start()

for thread in threads:
    thread.join() #only when all the threads are done then it will print the print('End of program.')
##############################################

print('End of program.')

achene
capsule
caryopsis
cypsela
fibrous drupe
follicle
legume
loment
nut
samara
schizocarp
silique
silicle
utricle
capsule
utricle
schizocarp
samara
achene
legume
fibrous drupe
nut
loment
cypsela
silicle
caryopsis
follicle
silique
schizocarpsamara
loment
nut
caryopsis
follicle
silique
fibrous drupe
utricle
achene

cypsela
legume
legume
samara
fibrous drupe
schizocarp
nut
achene
loment
follicle
cypsela
nut
schizocarp
cypsela
follicle
achene
loment
fibrous drupe
legume
schizocarpnut
cypsela
follicle
fibrous drupe
achene
loment
legume

cypsela
schizocarp
achene
nut
fibrous drupe
schizocarpfibrous drupe
cypsela

fibrous drupe
schizocarp
cypsela
fibrous drupe
End of program.


#### FA-2:
After creating one thread per message in FA-1, let's verify that 
all threads are truly started before we continue.

1) Right after creating each thread, store the thread object in 'threads'.
2) Then create a short loop that prints each thread's 'is_alive()' status.
3) Confirm they're all alive before you join them.

Expected outcome:
- A list of boolean values or lines like "Thread #0: True", "Thread #1: True", etc. 
  if the threads haven't finished yet.

In [6]:
## Your solution goes here: ##
## FA-2:
import threading
import time
dry_fruits=['achene',
    'capsule',
    'caryopsis',
    'cypsela',
    'fibrous drupe',
    'follicle',
    'legume',
    'loment',
    'nut',
    'samara',
    'schizocarp',
    'silique',
    'silicle', 
    'utricle' ]
threads = []
import random
def sayMessage(message, reps): 
    for _ in range(reps): 
        print(message)
        time.sleep(1)

for word in dry_fruits:
    t = threading.Thread(target = sayMessage, args = (word, random.randint(1,10)) )
    threads.append(t)
    

for thread in threads: #if we dont have this part with start - then the Main thread will be False=not alive
    thread.start()

for i, t in enumerate(threads): #this is the Main thread; #the results are printed one after another because we dont have sleep func in the main thread
    print(f"Thread #{i}: {t.is_alive()}")

for thread in threads:
    thread.join()

##############################

# Example usage:
# for i, t in enumerate(threads):
#     print(f"Thread #{i}: {t.is_alive()}")

achene
capsule
caryopsis
cypsela
fibrous drupe
follicle
legume
loment
nut
samara
schizocarp
silique
silicle
utricle
Thread #0: True
Thread #1: True
Thread #2: True
Thread #3: True
Thread #4: True
Thread #5: True
Thread #6: True
Thread #7: True
Thread #8: True
Thread #9: True
Thread #10: True
Thread #11: True
Thread #12: True
Thread #13: True
capsule
cypsela
achene
caryopsis
legume
follicle
schizocarp
silicle
samara
nut
utricle
loment
silique
achenecypsela
capsule

silicle
samara
loment
silique
schizocarp
nut
legume
utricle
follicle
capsulecypsela
achene

samara
loment
follicle
silicle
nut
schizocarp
silique
utricle
legume
cypsela
achene
loment
samara
schizocarp
nut
silique
utricle
legume
silicle
follicle
achene
legume
utricle
samara
nut
silique
silicle
follicle
loment
schizocarp
achene
follicle
loment
nut
utricle
samara
silique
follicleloment
nut
silique

samara
utricle
samarasilique

follicle
nut
nut
follicle


#### FA-3:
Add a small random sleep in 'sayMessage' for each iteration (e.g., time.sleep(random.uniform(0.1, 0.5)))
so messages interleave more unpredictably.

1) Modify 'sayMessage' or create a new function 'sayMessageRandom' that sleeps a random fraction of a second.
2) Create several threads using 'sayMessageRandom' with different messages.
3) Observe the interleaved output.

Expected outcome:
- The threads' outputs appear in more random order due to the varying sleep intervals.

In [8]:
## Your solution goes here: ##
## FA-3:
import random
def sayMessageRandom(message, reps): 
    for _ in range(reps): 
        print(message)
        time.sleep(random.uniform(0.1, 0.5))

t1 = threading.Thread(target=sayMessageRandom, args=("Hello concurrency!", 5))

t2 = threading.Thread(target=sayMessageRandom, args=("Hi!", 5))

t3 = threading.Thread(target=sayMessageRandom, args=("Yes!", 5))


t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()
##############################

# Example usage:
# t1 = threading.Thread(target=sayMessageRandom, args=("Hello concurrency!", 5))
# t1.start()
# t1.join()

Hello concurrency!
Hi!
Yes!
Hello concurrency!
Hi!
Hi!Yes!

Hi!
Hello concurrency!
Yes!
Hello concurrency!
Yes!
Hi!
Hello concurrency!
Yes!


### Class implementation

It is also possible to integrate threads (or processes) into a dedicated class. Below is an example of how you can use an object-oriented solution, by creating a class for your threads that inherits from Thread:

In [9]:
import threading

class myThread(threading.Thread): #we inherit from Thread class
  
    def __init__(self, message): 
        
        threading.Thread.__init__(self) #we call the initialization of Thread
        self.message = message #we can add anything here
        
    #We need to override the run method of Thread
    # This is the function that will be called by the thread method start
    def run(self): 
        for _ in range(20):
            print(self.message) #note access to the arguments of the target goes through class attributes

m = myThread("Hello, world!")
m.start() #use .start() to start a new thread. If you use .run() the method will not be executed in a different thread.
#start will automatically call the run() func we dont have to call run directly
print('we can run other code on the meantime')
m.join()
print('done')

Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!
we can run other code on the meantime
done


### FA-4:


The code below provides a function to download comics from XCKD. Create a class *ComicThread* and use it to download 10 comics concurrently (note that you don't need to modify *get_issue*).

In [10]:
#Note: you can install the libraries below with pip3 or conda
#However, sometimes Jupyter Notebooks do not see them.
#If you experience too much trouble importing the libraries, solve
#this exercise in an external environment (e.g. PyCharm, Spyder, ...)
#and then paste your solution here for submission.
import requests, shutil
from bs4 import BeautifulSoup as bs
import threading


# code adapted from:
# https://github.com/chavarera/python-mini-projects/blob/master/projects/XKCD_downloader/xkcd_dowloader.py
def get_issue(issue_number):
    
    url = "https://xkcd.com/"+ str(issue_number)

    response = requests.get(url)

    #Checking if we can fetch the url or not
    if response.status_code ==200:
        soup = bs(response.content, 'html.parser')
        image_link = soup.find_all('img')[1]['src']
        image_name = image_link.split('/')[-1]
        image_url = "https:" + image_link
        r = requests.get(image_url, stream=True)
        if r.status_code == 200:
            #This ensures the image file is loaded correctly
            r.raw.decode_content = True

            # Creating the image file
            with open(image_name, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

            print('Image successfully Downloaded: ', image_name)
        else:
            print('Image Couldn\'t be retreived')
    else:
        print("Issue number {} is invalid".format(issue))



## FA -4 ###############################################

class ComicThread(threading.Thread):
    def __init__(self, issue_number): 
        threading.Thread.__init__(self) #we dont know yet this init; instead of threading.Thread we can write super.__init__(self) to get the init of this other class
        self.issue_number=issue_number
        
    def run(self): 
        get_issue(self.issue_number)

threads=[]
for i in range(1,11): #we start from 1 because for 0 index it raises an error; this is a problem of the given resource not the code
    t=ComicThread(i) #i is the issue_number and ComicThread is threading.Thread
    threads.append(t)
    t.start()

for thread in threads:
    thread.join()

print('all comics are downloaded')


########################################################

Image successfully Downloaded:  landscape_cropped_(1).jpg
Image successfully Downloaded:  red_spiders_small.jpg
Image successfully Downloaded:  blownapart_color.jpg
Image successfully Downloaded:  tree_cropped_(1).jpg
Image successfully Downloaded:  barrel_cropped_(1).jpg
Image successfully Downloaded:  island_color.jpg
Image successfully Downloaded:  pi.jpg
Image successfully Downloaded:  firefly.jpg
Image successfully Downloaded:  irony_color.jpg
Image successfully Downloaded:  girl_sleeping_noline_(1).jpg
all comics are downloaded


#### FA-5:
Create 3 threads of 'ComicThread' to download 3 different comic issues 
(e.g., issues 101, 150, 200). Start them all and wait for them to complete. 
Print a final message "All 3 comics downloaded."

Expected outcome:
- The 3 images for issues 101, 150, and 200 are fetched concurrently, 
  with messages like "Image successfully Downloaded: <filename>" 
  appearing in unpredictable order.


In [11]:
## Your solution goes here: ##
## FA-5:
import threading
class ComicThread(threading.Thread):
    def __init__(self, issue_number): 
        threading.Thread.__init__(self) #we dont know yet this init; instead of threading.Thread we can write super.__init__(self) to get the init of this other class
        self.issue_number=issue_number
        
    def run(self): 
        get_issue(self.issue_number)

# t2 = threading.Thread(target=sayMessageRandom, args=("Hi!", 5))
t1=ComicThread(101)
t2=ComicThread(150)
t3=ComicThread(200)
    
t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print('all comics are downloaded')

##############################

# Example usage:
# t1 = ComicThread(101)
# t2 = ComicThread(150)
# t3 = ComicThread(200)
# ...
# print("All 3 comics downloaded.")

Image successfully Downloaded:  laser_scope.jpg
Image successfully Downloaded:  bill_nye.png
Image successfully Downloaded:  grownups.png
all comics are downloaded


---
## Thread communication

Threads "live" within the process that creates them, *i.e.*, they have access to the memory of their parent process. This means that threads can see the variables that are in the scope of the thread creation call. You can see this in the example below.

In [2]:
import threading
import time
    #greetandmessage is the side thread
def greetAndMessage(message, reps): #greetAndMessage is a side thread and it can access the greeting variable because it is a global
    time.sleep(2)
    print(greeting) #The thread has access to this variable, which lives in the memory of the parent process
    for _ in range(reps):
        print(message)

    

greeting = "Good Morning"
#We create a thread to run our function
t1 = threading.Thread(target = greetAndMessage, 
                      args = ("Hello, world!", 2)) #this is the Main Thread and it calls the greetAndMessage and makes greetAndMessage a thread so in the end t1 can access greeting
#start the thread
t1.start()
#wait until the thread ends to finish the program:
t1.join() 
print("End.")

#it is like triangle: greetandmessage -> greeting; t1->greetand message; t1->greeting

Good Morning
Hello, world!
Hello, world!
End.


Having direct access to the memory of the parent can be an advantage, but it may also have undesirable effects. The following example is likely to not produce the output you expect:

In [3]:
import threading
import time
#everything here is a main thread; the only side thread is the greetandmessage function defined above and called here inside t1 and t2
t1 = threading.Thread(target = greetAndMessage, 
                      args = ("Hello, world!", 2)) 

t2 = threading.Thread(target = greetAndMessage, 
                      args = ("Hello, world!", 2)) 

greeting="Good Morning" #main thread

#start thread 1, expecting to see Good Morning
t1.start() 

#because we dont have sleep() here, the good afternoon will run so fast after the good morning that it will overwrite good morning

#now another one for Good Afternoon
greeting = "Good Afternoon"  #main thread; this overwrites the good morning even though the 1st thread got started
t2.start()

#wait until the thread ends to finish the program
t1.join() 
t2.join()

Good Afternoon
Hello, world!
Hello, world!
Good Afternoon
Hello, world!
Hello, world!


For the example above, the solution is simple. 

### FA-6:

How would you re-organize the code in the example above so that t1 greets Good Morning?

In [14]:
#### FA-6 #####################
import threading
import time

t1 = threading.Thread(target = greetAndMessage, #one way is to add time.sleep() func; 2nd way is to add extra par the actual greeting: args = ("Hello, world!", 2,"Good Morning")) 
                      args = ("Hello, world!", 2)) 

t2 = threading.Thread(target = greetAndMessage, 
                      args = ("Hello, world!", 2)) 

greeting="Good Morning" #main thread

#start thread 1, expecting to see Good Morning
t1.start() 

time.sleep(4) 
#now another one for Good Afternoon
greeting = "Good Afternoon"  
t2.start()

#wait until the thread ends to finish the program
t1.join() 
t2.join()

##############################

Good Morning
Hello, world!
Hello, world!
Good Afternoon
Hello, world!
Hello, world!


#### FA-7:
Define a function 'threadWithGreeting(greet)' that returns a newly created 
Thread object which, when started, prints the greet string 3 times. 
Demonstrate by creating 2 or 3 threads with different greetings.

Expected outcome:
- Something like:
  Thread 1 => "Hello from T1"
  Thread 1 => ...
  Thread 2 => "Hola from T2"
  ...

In [15]:
## Your solution goes here: ##
## FA-7:

def myfunc(greet):
    for i in range(3):
        print(greet)

        
def threadWithGreeting(greet):
    return threading.Thread(target=myfunc, args=(greet,)) #each thread has its own greet since it is passed as an argument; so they dont dep on one and the same greet

##############################

# Example usage:
t1 = threadWithGreeting("Morning from T1")
t2 = threadWithGreeting("Evening from T2")
t1.start()
t2.start()
t1.join()
t2.join()

Morning from T1
Morning from T1
Morning from T1
Evening from T2
Evening from T2
Evening from T2


However, this can easily get complicated if threads depend on the values of shared variables. This becomes particularly delicate when threads use shared variables to communicate between them. That's the case in the next example.

The following code is written with the expectation that the first thread that executes f shows a greeting, and then sets it to an empty string such that the rest of threads don't greet again.

If you run it as is, it may likely produce the desired behavior. But if you uncomment the time.sleep, you will likely see a different behavior (if you don't, increase the argument to sleep). 

In [16]:
import threading, time

greeting='Hi' #1

def greet_once():  #5
    global greeting #6
    print(greeting) #7 we print the global; the 1st thread comes here and it is 'Hi'
    #time.sleep(2)
    greeting = '' #8 we change the global to empty str; the 2nd thread and the rest come here and be empty strings 
    print('other stuff that this function does') #9

threads = [] #2

for _ in range(10): #3 we create 10 threads but we havent started them yet
    threads.append(threading.Thread(target = greet_once))

for t in threads:
    t.start() #4 when i start the thread, it goes to the function greet_once();

    
for t in threads:
    t.join() #10


Hi
other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does

other stuff that this function does


### FA-8:

Why does the code (with uncommented time.sleep(2)) not produced the desired behavior?

In [ ]:
#### FA-8 #####################

fa8_answer = """
Because of the sleep() func, the 1st thread is sleeping for 2 seconds this makes the other threads to get started and print the greeting before reset the greeting to empty string.
"""

##############################

In [17]:
import threading, time

greeting='Hi' #1

def greet_once():  #5
    global greeting #6
    print(greeting) #7 we print the global; the 1st thread comes here and it is 'Hi'
    time.sleep(2)
    greeting = '' #8 we change the global to empty str; the 2nd thread and the rest come here and be empty strings 
    print('other stuff that this function does') #9

threads = [] #2

for _ in range(10): #3 we create 10 threads but we havent started them yet
    threads.append(threading.Thread(target = greet_once))

for t in threads:
    t.start() #4 when i start the thread, it goes to the function greet_once();

    
for t in threads:
    t.join() #10


Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does


#### FA-9:
We saw an example with greet_once() that modifies the global 'greeting'. 
Now let's add a second global variable 'mode' = 'happy' or 'serious'. 
Inside greet_once() we print something extra if mode=='happy'. 
Demonstrate how multiple threads can cause unexpected outputs 
if they simultaneously modify 'mode' without coordination.

Expected outcome:
- Potentially jumbled or inconsistent prints due to race conditions on 'mode'.

In [18]:
## Your solution goes here: ##
## FA-9:
import threading, time

greeting='Hi' 
mode = 'happy'
def greet_once():  
    global greeting
    print(greeting) #1st th print hi and goes to print('Happy greeting'); the 2nd and the rest threads print hi and go to the waiting
    global mode
    if mode == 'happy':
        print('Happy greeting')
        mode = 'serious'
    time.sleep(2) #when all the threads are here again the same process repeat but we start with serious because we changed it in the 1st func
    if mode == 'serious': 
        print('Serious Greeting')
        mode = 'happy'
    
    greeting = '' 
    print('other stuff that this function does') 
    
threads = [] 

for _ in range(10):
    threads.append(threading.Thread(target = greet_once))

for t in threads:
    t.start() 

    
for t in threads:
    t.join() 
##############################

# Example usage:
# greeting='Hello'
# mode='happy'
# ...


Hi
Happy greeting
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Serious GreetingSerious Greeting
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does

other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does


#### FA-10:
Use a threading.Lock to protect access to 'greeting' from the greet_once() example. 
This ensures only one thread at a time can modify or read 'greeting'. 
Verify that with the lock, only one actual greeting is printed.

Expected outcome:
- Exactly one greeting is printed. The other threads see greeting as empty or something else.

In [19]:
## Your solution goes here: ##
## FA-10:
import threading, time

greeting='Hi'
lock = threading.Lock()

def greet_once():  
    
    global greeting

    with lock: # when the first thread comes then it will lock it for the other threads until it comes out of the lock; every thread is locked when it comes here
        if greeting:
            print(greeting) 
            greeting = '' #this line change the greeting to empty str for every next thread after the 1st one

    print('other stuff that this function does') 
    
threads = [] 

for _ in range(10):
    threads.append(threading.Thread(target = greet_once))

for t in threads:
    t.start() 

    
for t in threads:
    t.join() 

##############################

# Example usage:
# lock = threading.Lock()
# ...
# def greet_once_locked():
#   with lock:
#       # read/modify greeting

Hi
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does


## Thread communication with queues

As you have seen in the course materials, a Queue is a useful datastructure for thread communication. The reason why we prefer to use Queues rather than lists is because Queues add some methods to ensure Thread Safety.

This is one example from the chapter you have read, with some added comments:

In [20]:
import threading
from queue import Queue     # (a,b,c,d,S) S is sentinel and it is always in the end of the queue and we take out each element in consequtive order;
                            #when we reach S -> end of the queue

SENTINEL = object() #this is an empty object, used just as a signal
          #q is a key letter representing queue -> we can use any other letter; n is how many numbers we want in the queue
def producer(q, n): #this function produces Fibonacci numbers
    a, b = 0, 1 #just because of the fibonacci task we need the 1st two nums to be spefic nums so we can sum them and create the 3rd num
    while a <= n: #continue adding a num in the fib sequence until the current num is <= the count of nums
        q.put(a)    #we add next fib number in the queue to the queue
        a, b = b, a + b 
    q.put(SENTINEL)
    
def consumer(q):
    while True:
        num = q.get()  #read from the queue
        q.task_done()  #every time we read from a queue with get, we should
                       #signal a task_done
                       #otherwise the element is blocked and .join() won't finish
        if num is SENTINEL:
            break
        print("Got number {}".format(num))
    
#We create one single queue
q = Queue()
#We create multiple threads with access to the queue
cns = threading.Thread(target=consumer, args=(q, )) #consumer does not need to know the end of the sequence; when Sentinel is met-> break 
prd1 = threading.Thread(target=producer, args=(q, 5)) #thi sis the input so we need to know where the of the seq is = 5


#We start the two threads
cns.start()
prd1.start()


#When all the the numbers from the queue have been processed
q.join() #!!! we write this without q.start() - this means that we took all the nums out
cns.join()
prd1.join()
print('Done')

Got number 0
Got number 1
Got number 1
Got number 2
Got number 3
Got number 5
Done


# FA-11:

Modify the example of *greet_once* by creating a new function *greet_once_queue* such that it uses a Queue instead of a global variable. 

In [5]:
###### FA - 11 ###############
import threading, time
from queue import Queue 

#because we use queue and we dont have lock but only if statement, only the 1st thread can take the 'Hi' and even if the 2nd thread is fast and it comes right after the 1st thread - it can still not take the 'Hi', so 'Hi' is taken once only
greeting_queue = Queue()
greeting_queue.put('Hi')

def greet_once_queue(greeting_queue):  
    try:
        greeting = greeting_queue.get_nowait() # When First thread comes, it will get the Hi from queue and now queue is empty. all the other thread will not be able to get anything out of the queue so printing Hi only once
    except:
        greeting = ''

   
    if greeting:  # it was printing empty spaces in greeting that's why I put a barrier of: if there is a greeting then print it otherwise no.
        print(greeting)
    # time.sleep(2)
    print('other stuff that this function does') 

threads = [] 

for _ in range(10): 
    threads.append(threading.Thread(target = greet_once_queue,args=(greeting_queue,)))

for t in threads:
    t.start() 


for t in threads:
    t.join() 

############################

Hi
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does


#### FA-12:
In 'greet_once_queue', after greeting once, enqueue (i.e., add) a short confirmation message 
like "Greeting complete!" to a 'messages' queue. 
Then in the main thread, read from 'messages' and print them all.

Expected outcome:
- We see exactly one "Hi" from greet, plus "Greeting complete!" 
  in the main thread after greet finishes.

In [6]:
## Your solution goes here: ## 

## FA-12: 
'''
#this is the solution that you expect: when we use two queues and we don't use global variable -> but this gives us a wrong answer because the queues overlap
import threading, time
from queue import Queue 

greeting_queue = Queue()
greeting_queue.put('Hi')

def greet_once_queue(q_messages): 
    try:
        greeting = greeting_queue.get_nowait() 
        print(greeting)
    except:
        greeting = ''
    if greeting:  
        q_messages.put(greeting)
        q_messages.put("Greeting complete!")
        
    # time.sleep(2)
    # print('other stuff that this function does') 

threads = [] 

for _ in range(10): 
    threads.append(threading.Thread(target = greet_once_queue,args=((greeting_queue,))))

for t in threads:
    t.start() 

for t in threads:
    t.join() 
############################
# Example usage:
q_messages = Queue()
greet_once_queue(q_messages)
while not q_messages.empty():
    print("From greet:", q_messages.get())

output:

Hi
Hi
Greeting complete!
Hi
Greeting complete!
Greeting complete!
Greeting complete!
Hi
Greeting complete!
Greeting complete!
Greeting complete!
From greet: Greeting complete!
From greet: Greeting complete!
'''
#This solution below is the solution when we use 1 queue and we use global variable which seems to be more correct because it gives the expected output

#enqueue (i.e., add) = adding sth to the que
#dequeue = delete 

import threading, time
from queue import Queue 

greeting  = "Hi"


def greet_once_queue(q_messages):  
    global greeting
    if greeting: 
        q_messages.put(greeting)
        q_messages.put("Greeting complete!")
    else:
        greeting = ""
    # time.sleep(2)
    print('other stuff that this function does') 

threads = [] 

for _ in range(10): 
    threads.append(threading.Thread(target = greet_once_queue,args=(greeting_queue,)))

for t in threads:
    t.start() 


for t in threads:
    t.join() 
    
q_messages = Queue()
greet_once_queue(q_messages)
while not q_messages.empty():
    print("From greet:", q_messages.get())
##############################

# Example usage:
# q_messages = Queue()
# greet_once_queue(q_messages)
# while not q_messages.empty():
#     print("From greet:", q_messages.get())

other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
other stuff that this function does
From greet: Hi
From greet: Greeting complete!


#### FA-13:
Spawn multiple threads that each call greet_once_queue. 
Give each thread a different greeting string via a queue.

1) Each thread pulls its greeting from a 'greeting_queue' 
2) The main thread enqueues multiple greetings, e.g. "Hello", "Hi", "Hey" 
3) Observe the single-greeting behavior in parallel threads.

Expected outcome:
- Each thread greets exactly once, producing messages in random interleaving.

In [23]:
## Your solution goes here: ##
## FA-13:
import threading, time
from queue import Queue 


def greet_once_queue(greeting_q):  
    if not greeting_q.empty(): #this step is needed when we have more threads than the greetings; but in our case, it is okay without this step too
        greeting = greeting_q.get() #once we get one greeting out, the list forgets it just like in generators
        greeting_q.task_done()
        print(f"Thread {threading.current_thread().name} says {greeting}")
    # else:
        # greeting = ""
    time.sleep(2)
    print('other stuff that this function does') 

# Example usage:
greeting_q = Queue()
for g_str in ["Hello", "Hi", "Hey"]:
    greeting_q.put(g_str)


threads = [] 
for _ in range(3): #  there were 3 greetings ["Hello", "Hi", "Hey"]. And we can't get the length dynamically since they were not stored into a variable
    t = threading.Thread(target=greet_once_queue, args=(greeting_q,))
    threads.append(t)
    t.start() #this we can do with a for loop too; the difference is that if we have for loop, then 1st the 3 threads will be created then started parallel
    #now we create one thread and start it, after we create the 2nd and we start it etc 
        
for thread in threads:
    thread.join()
##############################

Thread Thread-123 says Hello
Thread Thread-124 says Hi
Thread Thread-125 says Hey
other stuff that this function doesother stuff that this function does
other stuff that this function does



---
## Threading implementation of the Pot of Sweets

Now we are ready to implement a threading version of the Pot of Sweets. The queues are used here to communicate between the main process and the thread.

In [24]:
import time, threading, queue
from pcinput import getString


def addSweet(q,s):
    switch = True
    potOfSweets = 0
    while True:
        time.sleep(1)
        if not(s.empty()): #s is supposted to carry the signals from the user and if there is no signal word like 'stop', a new row 'added a sweet' will be printed
            sig = s.get()
            s.task_done()
            if sig == "on": #on continue printing sweets
                switch = True
            if sig == "off": #off stop printing sweets but the thread is still alive
                switch = False
            if sig == "stop": #this stops the thread and makes it not alive
                print("Stopping addSweet.\n")
                q.put(potOfSweets) #put the pile of sweets in the queue
                break
        if switch:
            print("added a sweet\n")
            q.put(1)
            
    print("Done!\n")
    
    return
    
# ---

#START POINT
#We start with the switch on
switch = True 

#We create two queues, to be used by addSweet
q = queue.Queue() #this queue will handle the amount of sweets in the pot
s = queue.Queue() #this queue will handle the control signals

#We create a thread for addSweet, with the queues as arguments
t = threading.Thread(target = addSweet, args = (q,s))

#Start the thread
t.start()

    
# This won't stop until the user sends a stop signal!
signal='on'
while signal != 'stop': #we have two stops this is the 1st
    if not t.is_alive():# this is the 2nd #when the thread is done, break
        break 
    
    #Prompt the user, asking what position the switch should have, and collect user's input in signal
    signal = getString("Insert switch position:")
    
    #Put user input in s queue
    s.put(signal) 
    
    
#Wait for signal cue to be fully processed
s.join()
#Wait for thread to end
t.join() 
    
#Count sweets in queue
total=0 #when the thread is ready, we come here and count how many things we have in the queue and we print the count at the end
while not q.empty():
    potOfSweets = q.get() #refresh the amount of sweets on the pile one last time.
    q.task_done()
    total+=potOfSweets

print("Total amount of sweets: ", total) #print the amount of sweets in the pile

#Wait for queue to be fully processed
q.join()

print('Program done.')



added a sweet

added a sweet

added a sweet

added a sweet

added a sweet



Insert switch position: on


added a sweet

added a sweet

added a sweet



Insert switch position: off
Insert switch position: on


added a sweet

added a sweet

added a sweet



Insert switch position: stop


Stopping addSweet.

Done!

Total amount of sweets:  11
Program done.


---
## Multiprocessing

Python provides the <code>multiprocessing</code> library to implement multiprocessing: https://docs.python.org/3/library/multiprocessing.html


Before we start, a note of caution. Since multiprocessing will create independent processes (which will be visible to your operating system), it is more sensible to compatibility issues. In this course, some students have experienced problems when using multiprocessing with Jupyter Notebooks on Windows. If you experience that, you may try one of these solutions:

* run this notebook from Jupyter Labs: https://jupyterlab.uvt.nl

* copy your code in a .py file and run it from a Python interpreter (e.g. with conda, with an IDE such as Pycharm or Spyder, or simply from your command line)

FYI, there exists a 3rd-party library called <code>multiprocess</code> which can be used as an alternative to multiprocessing, but we won't use it in this course. 


The creation of processes with the multiprocessing library is very similar to threading. This is the same example we used to introduce threading, but substituting the constructor from *threading.Thread* to *multiprocessing.Process*:

In [7]:
import multiprocessing
import time
#one process has multiple threads; different processes can run in parallel too
#  P1          P2      everything runs parallel
#t t t t     t t t t   everything runs parallel
def sayMessage(message, reps): 
    for _ in range(reps):
        print(message)
        time.sleep(1)

    
#We create a process to run our function
p1 = multiprocessing.Process(target = sayMessage, 
                      args = ("Hello, world!", 10)) 

#start the process
p1.start() 

print('We can run other instructions while the process is alive')


time.sleep(2) #pause the main program


print('The process keept running while the main program was sleeping')

#wait until the process ends
p1.join() 

#the following instruction is only printed after p1 has finished
print('end')

We can run other instructions while the process is alive
The process keept running while the main program was sleeping
end


### FA-14:

Create a class implementation of a process called *myProcess*, which inherits from *multiprocessing.Process*. This class should run a function that computes the square root of a number and prints it. Call it as shown below.

**Hint**: for this exercise, it is useful to look back at the example of a class implementation of a thread.

In [9]:
from math import sqrt #function to compute the square root
from pcinput import getString


###### FA-14 ###############################################

class myProcess(multiprocessing.Process):
    def __init__(self, number): 
        multiprocessing.Process.__init__(self) 
        self.number = number
    def run(self):
        result = (self.number)**(1/2)
        print("The square root of",self.number,"is:",result)
#when I run this cell in https://jupyterlab.uvt.nl this is the output but if I here the cell, the output is not what we expect
# start
# Insert a number: 4
# The square root of 4 is: 2.0
# Insert a number: 6
# The square root of 6 is: 2.449489742783178
# Insert a number: 25
# The square root of 25 is: 5.0
# Insert a number: stop
# done


#when I run the cell from the local environment the output is:
# start
# Insert a number: 4
# Insert a number: 6
# Insert a number: 25
# Insert a number: stop
# done
###########################################################

print('start')
pool = []
while True:
    n = getString("Insert a number:")
    if n == 'stop':
        break
    p=myProcess(int(n))
    p.start() #This time we start the process right away, so that we can keep prompting the user
    pool.append(p)

    
for p in pool:
    p.join()

print('done')


start


Insert a number: 4
Insert a number: 3
Insert a number: 6
Insert a number: 4
Insert a number: 3
Insert a number: 2
Insert a number: stop


done


#### FA-15:
Create a Process-based variant of 'sayMessage'. 
Call it 'SayProcess' that inherits from multiprocessing.Process 
and prints a message a specified number of times, 
sleeping 0.5s after each print.

Expected outcome:
- Running multiple SayProcess objects leads to interleaved or parallel prints 
  (depending on your CPU cores).

In [4]:
## Your solution goes here: ##
## FA-15:
import multiprocessing
import time
class SayProcess(multiprocessing.Process):
    def __init__(self, message, times): 
        multiprocessing.Process.__init__(self) 
        self.message = message
        self.times=times

    def run(self):
        for _ in range(self.times):
            print(self.message)
            time.sleep(0.5)

#this is the output I get when I run the code in  https://jupyterlab.uvt.nl

# Greetings from p1Salutations from p2

# Salutations from p2Greetings from p1

# Greetings from p1
##############################

# Example usage:
p1 = SayProcess("Greetings from p1", 3)
p2 = SayProcess("Salutations from p2", 2)
p1.start(); p2.start()
p1.join(); p2.join()

#### FA-16:
Now adapt 'myProcess' from FA-14 to store the result (the square root) 
in a multiprocessing.Queue instead of directly printing it. 
In the main process, gather the results from the queue after all processes finish.

Expected outcome:
- The main process prints a summary like "Roots: [1.4142, 3.1622, 5.0000, ...]"

In [6]:
## Your solution goes here: ##
## FA-16:
#The code loads infinitely here, but when I run it in https://jupyterlab.uvt.nl, it is different
# import multiprocessing
# class myProcess(multiprocessing.Process):
#     def __init__(self, number,q): 
#         multiprocessing.Process.__init__(self) 
#         self.number = number
#         self.q=q
#     def run(self):
#         result = (self.number)**(1/2)
#         self.q.put(result)
        
#This is the output I get when I run the code in  https://jupyterlab.uvt.nl
# 5.0
##############################

# Example usage:
# q_results = multiprocessing.Queue()
# p = myProcess(25, q_results)
# p.start()
# p.join()
# root = q_results.get() #because we dont have multiple things in the queue but only one, we dont need task_done()
# print(root)  # => 5.0

---
## Communication between processes

We have introduced a Queue as a method for communication between Threads. However, you may have noticed that the shared Queue was accessible to multiple Threads because it was in their memory scope, as Threads share the memory of the process that created them.

That's not the case for Processes. When you create a process, they get a **copy** of the memory of their parent. 

### FA-17:

Below is an adaptation of the code for producer-consumer, in which we use Threads instead of Processes. Run it and see what happens (you may want to stop the cell using the Stop button after a while). Why is this happening? Where is the code stuck?

In [7]:
# import multiprocessing
# from queue import Queue

# SENTINEL = object() #this is an empty object, used just as a signal

# def producer(q, n): #this function produces Fibonacci numbers
#     a, b = 0, 1
#     while a <= n:
#         q.put(a) #we add next fib number in the queue to the queue
#         a, b = b, a + b 
#     q.put(SENTINEL)
    
# def consumer(q):
#     while True:
#         num = q.get()  #read from the queue
#         q.task_done()  #every time we read from a queue with get, we should
#                        #signal a task_done
#                        #otherwise the element is blocked and .join() won't finish
#         if num is SENTINEL:
#             break
#         print(f'Got number {num}')
    
# #We create one single queue
# q = Queue()

# #Each process gets a copy of the queue
# cns = multiprocessing.Process(target=consumer, args=(q, ))
# prd1 = multiprocessing.Process(target=producer, args=(q, 5))
# prd2 = multiprocessing.Process(target=producer, args=(q, 10))


# cns.start()
# prd1.start()
# prd2.start()


# #When all the the numbers from the queue have been processed...:
# q.join()
# cns.join()
# prd1.join()
# prd2.join()
# print('Done')

#### FA-17 #############################


fa17_answer = """
This code gives an error or runs forever because the producer produces something, and the consumer does not have access to the producer queue.
Since each process has its own copy of the queue and the queues are not shared between the processes.
The code is stuck where the consumer is trying to get the things out of the queue, but does not find anything/Sentinel in the Queue to break the while loop, and it's running infinitely.
"""


######################################

#### FA-18:
Rewrite the example with producer/consumer from the 'Threads' version 
into a 'multiprocessing' version using 'multiprocessing.Queue'. 
Produce Fibonacci numbers up to 20 in one process, consume in another. 
Explain in a comment why it now won't get stuck.

Expected outcome:
- We see Fibonacci numbers up to 20 displayed, then 'Done'.

In [8]:
## Your solution goes here: ##
## FA-18:

import multiprocessing
  

SENTINEL = object()
          
def producer(q, n):
    a, b = 0, 1
    while a <= n:
        q.put(a)   
        a, b = b, a + b 
    q.put(SENTINEL)
    
def consumer(q):
    while True:
        num = q.get()   
        if num is SENTINEL:
            break
        print("Got number {}".format(num))

if __name__ == '__main__':
    q =  multiprocessing.Queue()
    cns = multiprocessing.Process(target=consumer, args=(q,))
    prd1 = multiprocessing.Process(target=producer, args=(q, 20))
    
    cns.start()
    prd1.start()
    
    cns.join()
    prd1.join()
    print('Done')

#this is the output I get when I run the code in  https://jupyterlab.uvt.nl    
# Got number 0
# Got number 1
# Got number 1
# Got number 2
# Got number 3
# Got number 5
# Got number 8
# Got number 13
# Got number <object object at 0x7f53d4617690>    
##############################

# Example usage:
# cns = multiprocessing.Process(target=consumer, args=(mp_queue,))
# ...

Done


#### FA-19:
Compare 'queue.Queue' vs. 'multiprocessing.Queue'. 
Why does the latter avoid the freeze issue we saw earlier?


In [2]:
## Your solution goes here: ##
## FA-19:

fa19_answer = """
Because queue.Queue is not shared between the processes, but multiprocessing. Queue is shared between the processes.
"""

##############################

As you have seen in the book chapter of this module, we need to use a different Queue, which is imported from the multiprocessing library. Note that this type of queue doesn't use a *task_done* method; for the rest, the interface is very similar to the Queue that you can import from the *queue* library. But behind the curtains, multiprocessing.Queue does an additional task: it takes care of transferring the contents of the Queue every time they change, so that each process can see the updated content in their portion of memory. Interestingly, for this to work smoothly, multiprocessing.Queue creates a special thread that takes care of the memory transfer :)

In [9]:
#Source: comm_queue_proc.py

import multiprocessing

SENTINEL = 'STOP'

def producer(q, n):
    a, b = 0, 1
    while a <= n:
        q.put(a)
        a, b = b, a + b
    q.put(SENTINEL)
    
def consumer(q):
    while True:
        num = q.get()
        if num == SENTINEL:
            break
        print(f'Got number {num}')
        
q = multiprocessing.Queue()
cns = multiprocessing.Process(target=consumer, args=(q, ))
prd = multiprocessing.Process(target=producer, args=(q, 35))

cns.start()
prd.start()

cns.join()
prd.join()

print('end')

end


## FA-20:

In this final formative assignment, you may use any of the techniques explained to you above to build your program (for instance, you can choose whether you want to use the class implementation version or not). 

The idea is to create a variant of the square root program you have created in FA-6. In this variant, instead of printing the square root right away, we want the program to print only in batches of 5, *i.e.*, the program should not print until there are at least 5 computed squared roots. The program needs to be constantly responsive to the user, *i.e.*, it should prompt for the next number for which the user wants a square root. The program only stops once the user inputs 'stop' (as in FA-14). 


Before writing this program, answer the following questions:

1. Which functionality should be parallelized?

2. Is it better to use threads or processes for this application? In which situation would you prefer one over the other?

2. How many different types of threads or processes will you need? What will be the main function of each one? And how many instances of the differnt types of processes do you expect to run at the same time?

3. Do your threads or processes need to communicate? If so, which data structure will you use? 


In [ ]:
# FA-20 ##################################################

#1.
fa20_answer_1 = """
The calculation of sqrt root should be parallelized, so when the program asks the user to put a number again,
the root can be calculated in the background while we are allowed to place another number without blocking.
"""


#2. 
fa20_answer_2 = """
Since the code is small and we calculate only the square root, it is better to use threads instead of processes because threads are lighter and we don't need to use processes. 
In the given case, it is also better to use Threads because the queue is shared because the queue makes it easier to batch the results via shared memory. 
If we the multiprocessing to share memory, then we need to use multiprocessing.Queue.
We would prefer to use Process when the data we need to process is large, e.g., image processing.
We would prefer to use Threads when the data is not that big.
"""


#3.
fa20_answer_3 = """
We need two Processes:
1. Main Process
2. Side Process
We need a main Process in which we keep asking the user, for example, numbers and sending them to the queue. 
And we need a side Process that consumes the numbers from the queue.
I am expecting:
1 main process for user input
1 side process that computes the square root while the main process is prompting the user.

"""


#4.
fa20_answer_4 = """
Yes, they need to communicate, and this happens through the shared queue. We store the results in a list data structure before printing.
"""


#Code:
import threading
import queue
from math import sqrt

inputq = queue.Queue()
resultq = queue.Queue()

SENTINEL = "STOP"

def sqrt_func(inq, outq):
    while True:
        num = inq.get()
        if num == SENTINEL:
            break
        result = sqrt(num)
        outq.put(result)
        inq.task_done()


t = threading.Thread(target=sqrt_func, args=(inputq, resultq))
t.start()

batch = []

while True:
    user_input = input("Give number: ")
    if user_input.lower() == "stop":
        inputq.put(SENTINEL)
        break
    try:
        number = float(user_input)
        inputq.put(number)
    except ValueError:
        print("Enter valid number!")

    while not resultq.empty():
        batch.append(resultq.get())
        if len(batch) == 5:
            print("Batch", [round(x, 4) for x in batch])
            batch.clear()


inputq.join()
t.join()

if batch:
    print("Remaining:", [round(x, 4) for x in batch])

print("Done.")


#this is the output:
# Give number:  4
# Give number:  5
# Give number:  7
# Give number:  3
# Give number:  5
# Give number:  3
# Batch [2.0, 2.2361, 2.6458, 1.7321, 2.2361]
# Give number:  stop
###########################################################

### Optional exercise

For practising purposes, implement the program suggested in the exercise above with an alternative approach (*e.g.*, if you used Threads, try a version with Processes).

You can use a timer to compare your solutions. Which one is faster? Would this change if instead of computing the squared root we ran a more costly operation?


# References

*  Martin Atzmueller (2019). Materials for Advanced Programming for CSAI: AP11 - Parallelism.ipynb.
*  Forbes, E. (2017). Learning concurrency in python : speed up your python code with clean, readable, and advanced concurrency techniques. Packt Publishing. 